In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time 

# DEFINE LOGIN CREDENTIALS
userID         = '1124570809'
client_id      = '9eb4c6ef069845e4bde31c6581daac26'
client_secret  = '9995ec38d7294435acfb7c8fe81b99ec'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# LOAD DATASETS IDS FROM CSV FILE
IDs_df = pd.read_csv("Data/DatasetsIDs.csv",";")
IDs_df

,Dataset,LINK,ID
0,0,https://open.spotify.com/playlist/2JyC6Os4Tjc4...,2JyC6Os4Tjc4V6LwXhJKst
1,1,https://open.spotify.com/playlist/2ZTUES6KnGqc...,2ZTUES6KnGqczd4RgAKuBf
2,2,https://open.spotify.com/playlist/2fvfIU3x4mvn...,2fvfIU3x4mvnx14hEt0VTx
3,3,https://open.spotify.com/playlist/6TuZ84F0r7VU...,6TuZ84F0r7VUfRTkHEc1x0
4,4,https://open.spotify.com/playlist/78mEtY1fJsU8...,78mEtY1fJsU81a1w8Ytxqa
5,5,https://open.spotify.com/playlist/61RvEJit9j8D...,61RvEJit9j8DMCcJTgrpo9
6,6,https://open.spotify.com/playlist/1Enng1Br4k2D...,1Enng1Br4k2DXSKdSnF7jF
7,7,https://open.spotify.com/playlist/5kktKb52Hkh1...,5kktKb52Hkh1jPFmWtAFEF
8,8,https://open.spotify.com/playlist/5DIZHlJmzmPM...,5DIZHlJmzmPM5PMXfIMAoT
9,9,https://open.spotify.com/playlist/1khlc5IqNuZb...,1khlc5IqNuZbsqiTWG1R17


In [25]:
# Load Playlists and Save into Data Folder

def getTrackIDs(username,playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks


# Function for retrieving track ids
def getTrackIDs_OLD(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

# Function that extracts track features
def getTrackFeatures(id):
  meta = sp.track(id)
  features = sp.audio_features(id)

  # meta
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  length = meta['duration_ms']
  popularity = meta['popularity']

  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']

  track = [name, album, artist, release_date, length, popularity, danceability, acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
  return track

# Loop through Datasets
for index, row in IDs_df.iterrows():
        ID   = row['ID']
        Name = row['Dataset']
        ids = getTrackIDs(userID, ID)
        print(ids)

        # loop over track ids 
        tracks = []
        for i in range(len(ids)):
          track = getTrackFeatures(ids[i])
          tracks.append(track)

        # create dataset
        df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])

        df.to_csv("Data/Datasets/Dataset" + str(Name) + ".csv", sep = ',')

    

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [3]:

def getTrackIDs(username,playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks


# Loop through Datasets
for index, row in IDs_df.iterrows():
    if index == 0:
        ID   = row['ID']
        Name = row['Dataset']
        ids = getTrackIDs(userID, ID)
        print(ids)

     

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

